In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.4 is required in this notebook
# Earlier 2.x versions will mostly work the same, but with a few bugs
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.4"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

**Custom loss function:**

Sin hiperparams

In [2]:
# como función (usar TF dentro en todo):
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = 1 * tf.abs(error) - 1**2 / 2
    # donde sea error chiquito, devuelve squared_loss, si no, devuelve linear_loss
    return tf.where(is_small_error, squared_loss, linear_loss)

# model.compile(loss=huber_fn, optimizer="nadam", metrics=["mae"])

# model.save("my_model_with_a_custom_loss.h5")

# model = keras.models.load_model("my_model_with_a_custom_loss.h5",
#                                 custom_objects={"huber_fn": huber_fn})

Con hiperparams (dos opciones, función normal (peor), subclass Loss (mejor))

In [3]:
# como función devuelta por otra función (threshold personalizable):
def create_huber(threshold = 1.0):
    # la misma funcion pero con threshold en vez del valor hard-coded 1
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error) / 2
        linear_loss = threshold * tf.abs(error) - threshold**2 / 2
        # donde sea error chiquito, devuelve squared_loss, si no, devuelve linear_loss
        return tf.where(is_small_error, squared_loss, linear_loss)

    return huber_fn

# model.compile(loss=create_huber(2.0), optimizer="nadam", metrics=["mae"])

# model.save("my_model_with_a_custom_loss_threshold_2.h5")

# model = keras.models.load_model("my_model_with_a_custom_loss_threshold_2.h5",
#                                 custom_objects={"huber_fn": create_huber(2.0)})

In [4]:
# como subclass de keras.losses.Loss:
class HuberLoss(keras.losses.Loss):
    # init
    def __init__(self, threshold = 1.0, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)
    # call, lo mismo que la funcion, pero usando self.threshold OJO!!!
    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss = self.threshold * tf.abs(error) - self.threshold**2 / 2
        # donde sea error chiquito, devuelve squared_loss, si no, devuelve linear_loss
        return tf.where(is_small_error, squared_loss, linear_loss)
    # para poder guardar los hiperparametros
    def get_config(self): # para poder guardar los hiperparametros
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}


# model.compile(loss=HuberLoss(2.), optimizer="nadam", metrics=["mae"])

# model.save("my_model_with_a_custom_loss_class.h5")

# al definir get_config se puede guardar el threshold automaticamente, y asi despues:

# model = keras.models.load_model("my_model_with_a_custom_loss_class.h5",
#                                 custom_objects={"HuberLoss": HuberLoss})

# eso usa el config (guardado con el modelo en el h5) y crea la loss correspondiente

# model.loss.threshold --> 2.0

**Custom metrics:**

Streaming metric (como la precision, que no sea simplemente la average de la metric en cada batch sino que tenga en cuenta toda la info necesaria del epoch para calcularla a medida que se avanza):

In [5]:
# subclass de keras.metrics.Mean, que es mejor que keras.metrics.Metrics para estas cosas
class HuberMetric(keras.metrics.Mean):

    def __init__(self, threshold = 1.0, name = "HuberMetric", dtype = None):
        self.threshold = threshold
        self.huber_fn = create_huber(threshold)
        super().__init__(name = name, dtype = dtype)

    def update_state(self, y_true, y_pred, sample_weight = None):
        metric = self.huber_fn(y_true, y_pred)
        # entiendo que esto es lo mismo que hacer super().update_state(...)
        # se encarga la clase Mean de hacer el update state y tener en cuenta las sample_weights
        super(HuberMetric, self).update_state(metric, sample_weight)

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

# model.compile(loss=keras.losses.Huber(2.0), optimizer="nadam", weighted_metrics=[HuberMetric(2.0)])

# sample_weight = np.random.rand(len(y_train))
# history = model.fit(X_train_scaled.astype(np.float32), y_train.astype(np.float32),
#                     epochs=2, sample_weight=sample_weight)

# model.save("my_model_with_a_custom_metric_v2.h5")

# model = keras.models.load_model("my_model_with_a_custom_metric_v2.h5",
#                                 custom_objects={"HuberMetric": HuberMetric})

**Ejercicio custom layers:**

In [12]:
class CustomLayerNormalization(keras.layers.Layer):

# epsilon es un hiperparametro
    def __init__(self, epsilon = 0.001, **kwargs):
        self.epsilon = epsilon
        super().__init__(**kwargs)

    def build(self, batch_input_shape):
        self.alpha = self.add_weight(name = "alpha",
                                     shape = batch_input_shape[-1:],
                                     dtype = tf.float32,
                                     initializer = "ones")
        
        self.beta = self.add_weight(name = "beta",
                                    shape = batch_input_shape[-1:],
                                    dtype = tf.float32,
                                    initializer = "zeros")
        super().build(batch_input_shape)

    def call(self, X):
        mean, variance = tf.nn.moments(X, axes = -1, keepdims = True)
        # OJO: sqrt(0) no tiene derivada, asi que mejor meter el self.epsilon DENTRO
        std = tf.sqrt(variance + self.epsilon)
        # ojo, usa siempre los self
        result = self.alpha*(X - mean)/std + self.beta
        return result
      
    def compute_output_shape(self, batch_input_shape):
        return batch_input_shape

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "epsilon": self.epsilon}

In [8]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

In [13]:
custom_layer_norm = CustomLayerNormalization()

keras_layer_norm = keras.layers.LayerNormalization()

X = X_train.astype(np.float32)

tf.reduce_mean(keras.losses.mean_squared_error(keras_layer_norm(X), custom_layer_norm(X)))

<tf.Tensor: shape=(), dtype=float32, numpy=1.0791172e-14>